Student's Name and Email Address

Boise State University, Department of Chemistry and Biochemistry

## CHEM 324: PChem Lab {-}
# Worksheet 5: Isothermal Titration Calorimetry of $MgCl_2$ and $CaCl_2$ with EDTA {-}

In [ ]:
# Import the main modules used in this worksheet
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression

In [ ]:
# Load the google drive with your files 
#from google.colab import drive
#drive.mount('/content/drive')
# The following needs to be the path of the folder with all your datafile in .csv format
path='DSC_Data/'#/content/drive/MyDrive/Colab Notebooks/ITC_Data/'

## Task 1: Indium Melting and Crystallization {-}
The element indium is often used as a calibration substance in DSC due to its low melting point of 156.6 &deg;C and its singular crystal structure. In your first DSC experiment you will determine the melting and solidification temperatures of indium, and the corresponding enthalpies. 

* Plot the peaks corresponding to the two processes, without the baselines, in the same figure.

* Determine the melting and crystallization temperatures from the curves. Determine the melting and crystallization enthalpies. Discuss potential differences between these quantities.

* Report the measured mass of your sample. Calculate the normalized and molar enthalpies of melting and solidification of indium

In [ ]:
# write your math here

* Using the following equation, $T_m=\frac{\Delta H_{m}}{\Delta S_{m}}$, alculate the molar entropy from the molar enthalpy of melting. 

In [ ]:
indium=pd.read_csv(path+'test indium.txt',header=1,delimiter='  ',usecols=[1,2,3,4],names=['time','heat-flow','Ts','Tr'])

In [ ]:
indium.head()

In [ ]:
plt.plot(indium['Tr']-indium['Ts'])

In [ ]:
melting=indium.iloc[200:400,:]
solidification=indium.iloc[500:700,:]

In [ ]:
plt.plot(melting['Ts'],melting['heat-flow'])
plt.plot(solidification['Ts'],solidification['heat-flow'])
plt.plot(np.ones(2)*156.6,[-5,7])

In [ ]:
from scipy.signal import medfilt
baseline=medfilt(melting['heat-flow'],151)
plt.plot(melting['Ts'],melting['heat-flow'])
plt.plot(melting['Ts'],baseline)

In [ ]:
from scipy.interpolate import PchipInterpolator
x = melting['Ts']
y = melting['heat-flow']-baseline
equispaced_x = np.linspace(np.min(x),np.max(x),1000)
interpolator = PchipInterpolator(x,y)
equispaced_y = interpolator(equispaced_x)
plt.plot(equispaced_x,equispaced_y)
plt.plot(x,y,'o',ms=1)

find a good way to integrate this curve

In [ ]:
np.sum((melting['heat-flow']-baseline))*(melting['Ts'].values[-1]-melting['Ts'].values[0])/200

## Task 2: Phase Transitions in Paracetatmol (a.k.a. Acetominophene) {-}

In your second experiment you have looked at the phase transitions in one of the most popular drugs. Paracetamol exhibit three non morphisms (forms I-III). You performed two melting runs on the pristine sample and on a melted sample that was cooled at room temperature. 

* Plot the two runs and compare the two different heating curves. 

* Determine the melting temperature and the melting enthalpy for all the transitions in the two runs. 

* Report the mass of your sample. Calculate the normalized (in kJ/g) and molar (in kJ/mol) enthalpies. 

In [ ]:
# your math here

* Using the following equation $T_m=\frac{\Delta H_{m}}{\Delta S_{m}},$ compute the molar entropy from the molar enthalpy of melting. 

* Discuss why the two spectra look different. Assuming that the first spectrum is from a crystalline sample (i.e. one of the three polymorphs of paracetamol), determine which form you started with. 

YOUR COMMENTS HERE

## Task 3: Glass Transition in PET {-}

In your last experiment you have looked at the glass transition of a common polymer. Polyethylene terephthalate (PET) is a polymer that is widely used in packaging, and exhibits both amorphous and crystalline regions and a characteristic glass transition temperature.

* Plot the DSC curve for PET. 

* Determine the glass transition temperature. 

## Task 4: Discussion {-}

* Add your results and the correspnding literature values in the following table.

* Discuss on the possible sources of discrepancy between your experiments and the literature. 

YOUR COMMENTS HERE

* In what field is DSC would be useful? Give 2-3 examples.

YOUR COMMENTS HERE

In [ ]:
# This cell is used to allow Google Colab to install the tools to convert the notebook to a pdf file
# Un-comment the following lines when you are ready to export the pdf 
#!apt-get install texlive texlive-xetex texlive-latex-extra pandoc
#!pip install pypandoc

In [ ]:
# Use this command to convert the finished worksheet into a pdf 
# NOTE : you may want to change the path of the file, if you are working in a different folder of the Google Drive
#!jupyter nbconvert --no-input --to PDF "./ITC_Worksheet.ipynb"